In [2]:
import cv2
import numpy as np
import pandas as pd

In [3]:
def findChildContours(frame, frame_count):
    contours, hierarchy = cv2.findContours(frame, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    child_contours = []
    
    hierarchy = hierarchy[0] 
    for i, c in enumerate(contours):
        # Return only innermost contours with no child contours 
        if hierarchy[i][2] < 0 and hierarchy[i][3] < 0:
            child_contours.append(c)
    x_coords = []
    y_coords = []
    size = []
    for c in child_contours:
        if cv2.contourArea(c) > 240: # Only save large contours # Originally 200
            m = cv2.moments(c)
            # Find Contour centre 
            x = m['m10'] / m['m00']
            y = m['m01'] / m['m00']
            x_coords.append(x)
            y_coords.append(y)
            size.append(cv2.contourArea(c))
    frame_counts = [frame_count] * len(x_coords) # Which frame these contours found in
    return list(zip(x_coords, y_coords, frame_counts)) # Zip lists to list of tuples # Size removed

In [21]:
path = 'video-to-frames/snippet.mp4' # using a random file on my laptop to test
cap = cv2.VideoCapture(path)
frame_width = 680
frame_height = 480
fps = 30.0

cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # fourcc API is case sensitive

# MAIN ISSUE WAS HERE ...
# https://answers.opencv.org/question/198205/how-to-save-thresholding-video-in-python-opencv/
# VideoWriter has a parameter for isColor (bool, default = True)
# for single-channel grayscale we need this set to False
out1 = cv2.VideoWriter('thresholded.mp4', fourcc, fps, size, False) 
out2 = cv2.VideoWriter('framediff.mp4', fourcc, fps, size, False)
out3 = cv2.VideoWriter('grayscale.mp4', fourcc, fps, size, False)
out4 = cv2.VideoWriter('grayblur.mp4', fourcc, fps, size, False)

# Type font for adding frame counter to video
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

delay_time = 1 # Delay next loop for easier viewing
prev_frame = None 
counter = 0 # Frame counter
potential_waggles = [] # List to save potential waggles

while cap.isOpened():
    counter += 1
    ret, frame = cap.read()
    # Break when video ends
    if ret is False:
        break

    # Threshold Image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (21,21), 0)
    thresh = cv2.threshold(gray_blur, 108, 230, cv2.THRESH_BINARY)[1]
    
    # If first frame, set current frame as prev_frame
    if prev_frame is None:
        prev_frame = thresh
    current_frame = thresh

    # Background Subtraction and Find Contours within image
    frame_diff = cv2.absdiff(current_frame, prev_frame) 
    _, hierarchy = cv2.findContours(frame_diff, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    
    # Catch blank images and skip frame
    if hierarchy is None:
        continue
    else:
        find_waggles = findChildContours(frame_diff, counter)
        potential_waggles += find_waggles
    
    # Frame Counter
    cv2.putText(thresh, str(counter), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (150,0,0), 2)
    
    #create the files
    
    out1.write(thresh)
    out2.write(frame_diff)
    out3.write(gray)
    out4.write(gray_blur)  
    
    # Display the resulting frame
    cv2.imshow('Thresholded', thresh)
    cv2.imshow('Frame Diff', frame_diff)
    cv2.imshow('Grayscale', gray)
    cv2.imshow('GrayBlur', gray_blur)
    cv2.waitKey(delay_time)
    
    
    # Make current frame the previous frame for the next loop
    prev_frame = current_frame
    
    # q to quit
    if cv2.waitKey(delay_time) & 0xFF == ord('q'):
        break

    
cap.release()
out1.release()
out2.release()
out3.release()
out4.release()
cv2.destroyAllWindows()
#cv2.waitKey(1) # this doesn't seem to be needed, code runs fine without it